In [1]:
import pandas as pd
import regex as re

In [2]:
pd.set_option('display.max_row', 1000)
pd.set_option('display.max_columns', 50)

# Reading in the csv as dataframes
naming conventions chosen by Matthew

In [3]:
onboard_df = pd.read_csv('../data/VehicleDiagnosticOnboardData.csv')
onboard_df.head()
#in long form, may need to pivot remember the data is organized. faultid=recordid?

,Id,Name,Value,FaultId
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1


In [4]:
fault_df = pd.read_csv('../data/J1939Faults.csv')
fault_df.head(20)

C:\Users\zylst\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000
5,6,990431,2015-02-21 11:40:22.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1417,105438630,33.043564,-96.179722,2015-02-21 11:40:59.000
6,7,990439,2015-02-21 11:40:52.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1597,105344243,36.902916,-86.436481,2015-02-21 11:41:29.000
7,8,990441,2015-02-21 11:40:22.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00022630*082113134117*07700053*I0*BBZ*,79463845,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1429,105356054,38.228796,-84.582500,2015-02-21 11:41:44.000
8,9,990442,2015-02-21 11:40:22.000,High (Severity Low) Water In Fuel Indicator,NaN,04993120*00022630*082113134117*07700053*I0*BBZ*,79463845,6X1u10D1500000000,CMMNS,0,97,15,True,1,NaN,1429,105356054,38.228796,-84.582500,2015-02-21 11:41:44.000
9,10,990446,2015-02-21 11:41:55.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,17,False,1,NaN,1417,105438630,33.039953,-96.182592,2015-02-21 11:41:51.000


# Cleaning fault data

##### From the readme: 
<br> "Remove faults occurring in the vicinity of the service locations at (36.0666667, -86.4347222), (35.5883333, -86.4438888), and (36.1950, -83.174722)" <br> <br>


We want to go with 3 decimal points to get within 111m.

In [5]:
fault_df[['Latitude', 'Longitude']] = fault_df[['Latitude','Longitude']].astype(str)

In [6]:
fault_df['Latitude'] = fault_df['Latitude'].str.extract(r'(\-?\d+.\d{3})')

In [7]:
fault_df['Longitude'] = fault_df['Longitude'].str.extract(r'(\-?\d+.\d{3})')

In [8]:
fault_df['Location'] = fault_df['Latitude'].map(str) + ' , ' + fault_df['Longitude'].map(str)

In [9]:
fault_df = fault_df.loc[(fault_df.Location != '36.066 , -86.434') & (fault_df.Location != '35.588 , -86.443') & (fault_df.Location != '36.195 , -83.174')]

In [10]:
fault_df.shape

(1139316, 21)

##### From the readme:
<br> "Remove faults where the EquipmentID has more than 5 characters." confirmed with Michael to also drop NaNs

In [11]:
#confirmed that NaN values are not necessary
fault_df['EquipmentID'] = fault_df['EquipmentID'].dropna()

In [12]:
fault_df = fault_df[fault_df['EquipmentID'].str.len()<=5] 

In [13]:
fault_df.shape

(61682, 21)

In [14]:
fault_df['EquipmentID'].str.len().unique()

array([4, 3, 5], dtype=int64)

##### our own analysis:
<br> actionDescription,faultValue,ecuSource were all empty columns that we'll remove.

In [1]:
fault_df = fault_df.drop(['actionDescription','faultValue','ecuSource'],axis=1)

NameError: name 'fault_df' is not defined

In [16]:
fault_df.shape

(61682, 18)

# Pivoting onboard data from long to wide

In [17]:
#see what we're working with
onboard_df.head(15)

,Id,Name,Value,FaultId
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1
5,6,AcceleratorPedal,0,1
6,7,IntakeManifoldTemperature,78.8,1
7,8,FuelRate,0,1
8,9,FuelLtd,12300.907429328,1
9,10,EngineRpm,0,1


In [18]:
#FaultId is what we'll want to eventually join on

**onboard_df['FaultId'] <-> fault_df['RecordID']**

### onboard data dictionary
* **Id**: similar to index and not necessary for our data
* **Name**: Variable being measured at the time of whatever fault number FaultId is
* **Value**: the actual measurement value for whatever variable is being measured in Name
* **FaultId**: The same as the fault data csv's RecordID 

In [19]:
#chunking the data so I can test my code quickly
test_df = onboard_df.iloc[0:1000]

In [20]:
test_df.pivot(index='FaultId',columns='Name',values='Value')

Name,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
FaultId,,,,,,,,,,,,,,,,,,,,,,,
1,0,14.21,False,66.48672,423178.7,100.4,11,0,96.74375,0,1632.2,43.2,12300.907429328,0,NaN,False,78.8,1023,True,0,3276.75,NaN,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN
6,48,14.4275,False,64.6226,470381.4,181.4,30,38.28,196.5313,1514.5,9480,44,70349.809963756,4.583399,NaN,True,111.2,1023,NaN,13.6022,3276.75,NaN,6.67
7,82.8,14.2825,False,64.6226,278736.7,188.6,80,39.44,210.0313,1711.375,6292.2,64.8,40961.065436834,14.29175,NaN,True,78.8,1023,NaN,41.53478,3276.75,NaN,20.59
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1023,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1023,NaN,NaN,NaN,NaN,NaN


In [21]:
onboard_df = onboard_df.pivot(index='FaultId',columns='Name',values='Value')

In [22]:
onboard_df = onboard_df.reset_index()
onboard_df.columns.name = " "

In [23]:
onboard_df.head()

,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,0,14.21,False,66.48672,423178.7,100.4,11,0,96.74375,0,1632.2,43.2,12300.907429328,0,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN


# Merging the two tables for EDA

In [24]:
onboard_df.shape

(1187335, 25)

In [25]:
fault_df.shape

(61682, 18)

In [27]:
joined = onboard_df.merge(fault_df,how='outer',left_on='FaultId',right_on='RecordID')

In [29]:
joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1187335 entries, 0 to 1187334
Data columns (total 43 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   FaultId                    1187335 non-null  int64  
 1   AcceleratorPedal           531889 non-null   object 
 2   BarometricPressure         585976 non-null   object 
 3   CruiseControlActive        574916 non-null   object 
 4   CruiseControlSetSpeed      576458 non-null   object 
 5   DistanceLtd                585819 non-null   object 
 6   EngineCoolantTemperature   586071 non-null   object 
 7   EngineLoad                 585621 non-null   object 
 8   EngineOilPressure          586244 non-null   object 
 9   EngineOilTemperature       583912 non-null   object 
 10  EngineRpm                  586921 non-null   object 
 11  EngineTimeLtd              581366 non-null   object 
 12  FuelLevel                  502795 non-null   object 
 13  FuelLtd     